In [8]:
import pandas as pd
import numpy as np

min_reserve_price = 900
max_reserve_price = 1000
sell_price = 1000
total_fish = 1  # Total number of goldfish, can be anything.

def calculate_b(a):
    # Calculate 'b' based on 'a' ensuring the total area under the linear distribution equals 1
    return 2 * (1 - 50 * a) if a < 0.01 else 0.01

def expected_trades(bid, a, b):
    # Expected number of trades for a given bid, based on 'a' and 'b'
    slope = (b - a) / 100
    cdf = a + slope * (bid - min_reserve_price)
    return cdf * total_fish

columns = ['a', 'b', 'Optimal First Bid', 'Optimal Second Bid', 'Max Profit']
results_df = pd.DataFrame(columns=columns)

# Simulate for range of a
a_values = np.linspace(0, 0.01, 1000)  # 20 values of 'a' between 0 and 0.01

for a in a_values:
    b = calculate_b(a)
    max_profit = 0
    optimal_first_bid = 0
    optimal_second_bid = 0

    for first_bid in range(min_reserve_price, max_reserve_price + 1):
        first_bid_trades = expected_trades(first_bid, a, b)
        first_bid_profit = first_bid_trades * (sell_price - first_bid)

        for second_bid in range(first_bid + 1, max_reserve_price + 1):
            additional_trades = expected_trades(second_bid, a, b) - first_bid_trades
            additional_profit = additional_trades * (sell_price - second_bid)

            total_profit = first_bid_profit + additional_profit
            if total_profit > max_profit:
                max_profit = total_profit
                optimal_first_bid = first_bid
                optimal_second_bid = second_bid

    new_row = pd.DataFrame({
        'a': [a], 
        'b': [b], 
        'Optimal First Bid': [optimal_first_bid], 
        'Optimal Second Bid': [optimal_second_bid], 
        'Max Profit': [max_profit]
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)

display(results_df)


C:\Users\062n\AppData\Local\Temp\ipykernel_2104\2315461307.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


,a,b,Optimal First Bid,Optimal Second Bid,Max Profit
0,0.00000,2.000000,933,966,66.660000
1,0.00001,1.998999,933,966,66.626974
2,0.00002,1.997998,933,966,66.593947
3,0.00003,1.996997,933,966,66.560921
4,0.00004,1.995996,933,966,66.527895
...,...,...,...,...,...
995,0.00996,1.004004,932,966,33.798825
996,0.00997,1.003003,932,966,33.765819
997,0.00998,1.002002,932,966,33.732812
998,0.00999,1.001001,932,966,33.699806


In [9]:
first_bid_counts = results_df['Optimal First Bid'].value_counts().sort_index()

second_bid_counts = results_df['Optimal Second Bid'].value_counts().sort_index()

first_bid_occurrences = list(first_bid_counts.items())
second_bid_occurrences = list(second_bid_counts.items())

first_bid_occurrences, second_bid_occurrences

([(900, 1), (932, 4), (933, 995)], [(901, 1), (966, 858), (967, 141)])

In [10]:
# Adjust the simulation to consider non-integer bid values (i.e., in 0.1 increments)

# Initialize an empty DataFrame for the updated results
columns = ['a', 'b', 'Optimal First Bid', 'Optimal Second Bid', 'Max Profit']
updated_results_df = pd.DataFrame(columns=columns)

for a in a_values:
    b = calculate_b(a)
    max_profit = 0
    optimal_first_bid = 0
    optimal_second_bid = 0

    # Consider non-integer bids by using np.arange with a step of 0.1
    for first_bid in np.arange(min_reserve_price, max_reserve_price + 0.1, 0.1):
        first_bid_trades = expected_trades(first_bid, a, b)
        first_bid_profit = first_bid_trades * (sell_price - first_bid)

        for second_bid in np.arange(first_bid + 0.1, max_reserve_price + 0.1, 0.1):
            additional_trades = expected_trades(second_bid, a, b) - first_bid_trades
            additional_profit = additional_trades * (sell_price - second_bid)

            total_profit = first_bid_profit + additional_profit
            if total_profit > max_profit:
                max_profit = total_profit
                optimal_first_bid = first_bid
                optimal_second_bid = second_bid

    # Create a new row for the current 'a' value and append it to the DataFrame
    new_row = pd.DataFrame({
        'a': [a], 
        'b': [b], 
        'Optimal First Bid': [optimal_first_bid], 
        'Optimal Second Bid': [optimal_second_bid], 
        'Max Profit': [max_profit]
    })
    updated_results_df = pd.concat([updated_results_df, new_row], ignore_index=True)

# Count the optimal first bids under 933
optimal_first_under_933 = updated_results_df[updated_results_df['Optimal First Bid'] < 933]['Optimal First Bid'].count()

# Count the optimal second bids under 966 and then 967
optimal_second_under_966 = updated_results_df[updated_results_df['Optimal Second Bid'] < 966]['Optimal Second Bid'].count()
optimal_second_under_967 = updated_results_df[updated_results_df['Optimal Second Bid'] < 967]['Optimal Second Bid'].count()

optimal_first_under_933, optimal_second_under_966, optimal_second_under_967


C:\Users\062n\AppData\Local\Temp\ipykernel_2104\2361083902.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_results_df = pd.concat([updated_results_df, new_row], ignore_index=True)


(253, 1, 1000)

In [12]:
optimal_first_under_934 = updated_results_df[updated_results_df['Optimal First Bid'] < 934]['Optimal First Bid'].count()
optimal_first_under_934

1000

In [13]:
updated_results_df

,a,b,Optimal First Bid,Optimal Second Bid,Max Profit
0,0.00000,2.000000,933.3,966.6,66.666600
1,0.00001,1.998999,933.3,966.6,66.633567
2,0.00002,1.997998,933.3,966.6,66.600535
3,0.00003,1.996997,933.3,966.6,66.567502
4,0.00004,1.995996,933.3,966.6,66.534469
...,...,...,...,...,...
995,0.00996,1.004004,932.6,966.3,33.802093
996,0.00997,1.003003,932.6,966.3,33.769072
997,0.00998,1.002002,932.6,966.3,33.736050
998,0.00999,1.001001,932.6,966.3,33.703029


In [19]:
import pandas as pd
import numpy as np

# Parameters
min_reserve_price = 900
max_reserve_price = 1000
sell_price = 1000
total_fish = 100000
second_bid = 967  # Fixed second bid

def calculate_b(a):
    return 2 * (1 - 50 * a) if a < 0.01 else 0.01

def expected_trades(bid, a, b):
    slope = (b - a) / 100
    cdf = a + slope * (bid - min_reserve_price)
    return cdf * total_fish

# Simulate for a range of 'a' values
a_values = np.linspace(0, 0.01, 1000)
results_list = []  # Use a list to collect data for each simulation

for a in a_values:
    b = calculate_b(a)
    max_profit = -np.inf
    best_first_bid = None

    for first_bid in range(932, 935):  # Integer bids between 932 and 934
        # Calculate profit from the first bid
        first_bid_trades = expected_trades(first_bid, a, b)
        first_bid_profit = first_bid_trades * (sell_price - first_bid)

        # Calculate additional profit from the second bid
        second_bid_trades = expected_trades(second_bid, a, b) - first_bid_trades
        second_bid_profit = second_bid_trades * (sell_price - second_bid)

        total_profit = first_bid_profit + second_bid_profit
        if total_profit > max_profit:
            max_profit = total_profit
            best_first_bid = first_bid

    # Store results for this 'a' value in the list
    results_list.append({'a': a, 'b': b, 'Best First Bid': best_first_bid, 'Profit from Best First Bid': max_profit})

results_df = pd.DataFrame(results_list)

# Display the DataFrame
display(results_df)


,a,b,Best First Bid,Profit from Best First Bid
0,0.00000,2.000000,933,6.666000e+06
1,0.00001,1.998999,933,6.662697e+06
2,0.00002,1.997998,933,6.659395e+06
3,0.00003,1.996997,933,6.656092e+06
4,0.00004,1.995996,933,6.652789e+06
...,...,...,...,...
995,0.00996,1.004004,933,3.379881e+06
996,0.00997,1.003003,933,3.376578e+06
997,0.00998,1.002002,933,3.373275e+06
998,0.00999,1.001001,933,3.369973e+06


In [22]:
under_933 = results_df[results_df['Best First Bid'] <= 933]['Best First Bid'].count()
under_933

5

In [21]:
import pandas as pd
import numpy as np

# Parameters
min_reserve_price = 900
max_reserve_price = 1000
sell_price = 1000
total_fish = 100000  # Updated total number of fish
second_bid = 967  # Fixed second bid

def calculate_b(a):
    return 2 * (1 - 50 * a) if a < 0.01 else 0.01

def expected_trades(bid, a, b):
    if bid <= min_reserve_price:
        return 0
    slope = (b - a) / (max_reserve_price - min_reserve_price)
    cdf = a + slope * (bid - min_reserve_price - 1)  # Adjusted to consider only bids > barrier price
    return cdf * total_fish

# Simulate for a range of 'a' values
a_values = np.linspace(0, 0.01, 1000)  # More granular simulation with 1000 'a' values
results_list = []

for a in a_values:
    b = calculate_b(a)
    max_profit = -np.inf
    best_first_bid = None

    for first_bid in range(932, 935):  # Integer bids between 932 and 934
        first_bid_trades = expected_trades(first_bid, a, b)
        first_bid_profit = first_bid_trades * (sell_price - first_bid)

        second_bid_trades = expected_trades(second_bid, a, b) - first_bid_trades
        second_bid_profit = second_bid_trades * (sell_price - second_bid)

        total_profit = first_bid_profit + second_bid_profit
        if total_profit > max_profit:
            max_profit = total_profit
            best_first_bid = first_bid

    results_list.append({'a': a, 'b': b, 'Best First Bid': best_first_bid, 'Profit from Best First Bid': max_profit})

results_df = pd.DataFrame(results_list)

# To display the DataFrame in a non-interactive environment, use print instead of display
display(results_df)


,a,b,Best First Bid,Profit from Best First Bid
0,0.00000,2.000000,934,6.534000e+06
1,0.00001,1.998999,934,6.530763e+06
2,0.00002,1.997998,934,6.527526e+06
3,0.00003,1.996997,934,6.524289e+06
4,0.00004,1.995996,934,6.521052e+06
...,...,...,...,...
995,0.00996,1.004004,933,3.313280e+06
996,0.00997,1.003003,933,3.310045e+06
997,0.00998,1.002002,933,3.306810e+06
998,0.00999,1.001001,933,3.303575e+06
